<a href="https://colab.research.google.com/github/ewjohn127/dsc_project_1/blob/main/Phase1_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import csv
import sqlite3

# Idea 1 - Proof of Concept Model - Choose Genre With Highest Profitability And Lowest Budget

## Looking At:
* Genre Profitability
* Profitability v Movie Budget
* Example of Movies Fitting This Model

## Functions For Idea 1

In [3]:
'''
Takes a list of professions and checks to see if 'director' is in the list
'''

def identify_director(professions):
    if 'director' in professions:
        return True
    else:
        return False

In [4]:
'''
Takes a USD string and drops commas, units and converts to type int
'''

def clean_money(budget):
    split_list = budget.split(',')
    split_list[0] = split_list[0][1:]
    answer = ''
    for part in split_list:
        answer += part

    return int(answer)

In [5]:
'''
Takes an integer, converts to str, adds commas and $ sign
'''

def dirty_money(dollar_int):
    dollar_str = str(dollar_int)[::-1]
    final_dollar = ''
    counter = 0
    for number in dollar_str:
        counter += 1
        final_dollar += number
        if counter % 3 == 0:
            final_dollar += ','
    if final_dollar[::-1][0] == ',':
        final_dollar = final_dollar[:-1]
        return '$' + final_dollar[::-1]
    else:
        return '$' + final_dollar[::-1]

In [6]:
'''
Takes an integer
'''

def find_main_genre(genre_list):
    if len(genre_list) > 1:
        main_genre = genre_list[0] + '/' + genre_list[1]
    else:
        main_genre = genre_list[0]

    return main_genre

## Preparing Datasets

### Preparing Movie Budget Dataset

In [7]:
# Read In Dataset
df_budget = pd.read_csv('data/tn.movie_budgets.csv')

# Add New Columns With $$ As Int
df_budget['budget_int'] = df_budget['production_budget'].map(clean_money)
df_budget['total_gross_int'] = df_budget['domestic_gross'].map(clean_money) + df_budget['worldwide_gross'].map(clean_money)

# Drop Movies That Made No Money
df_budget1 = df_budget[df_budget['total_gross_int'] != 0].copy(deep=True)

# Add Gross Margin And Release Year Columns
df_budget1['gross_margin'] = round((df_budget1['total_gross_int'] - df_budget1['budget_int']) / df_budget1['total_gross_int'] * 100 , 2)
df_budget1['release_year'] = df_budget1['release_date'].map(lambda date_: int(date_[-4:]))

# Drop Unneeded Columns
df_budget_clean = df_budget1.drop(['id','release_date','production_budget','domestic_gross','worldwide_gross'],axis = 1)

# Sort By Year And Gross Margin
df_budget_sorted = df_budget_clean.sort_values(['release_year','gross_margin'],ascending=False)

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Preparing IMDB Title Basics Dataset

In [ ]:
# Read In Dataset
df_basics = pd.read_csv('data/imdb.title.basics.csv')

# Drop Nan Values In Genres
df_basics.dropna(subset=['genres'],inplace=True)

# Add genre_lists Column
df_basics['genre_lists'] = df_basics['genres'].map(lambda genres: genres.split(','))

# Add main_genre Column
df_basics['main_genre'] =  df_basics['genre_lists'].map(find_main_genre)

# Drop Unneeded Columns
df_basics.drop(['original_title','start_year','genres'],inplace=True, axis= 1)

# Change primary_title Column To movie
df_basics.rename({"primary_title":'movie'},axis=1,inplace=True)

### Preparing IMDB Ratings Dataset

In [ ]:
# Read In Dataset
df_ratings = pd.read_csv('data/title.ratings.tsv', delimiter='\t')

# Drop Unneeded Columns
df_ratings.drop(['numVotes'], inplace=True, axis= 1)\

# Cast averageRating To float
df_ratings['averageRating'] = df_ratings['averageRating'].astype(float)

### Preparing IMDB AKAS Dataset

In [ ]:
# Read In Dataset
df_akas = pd.read_csv('data/imdb.title.akas.csv')

# Drop Unneeded Columns
df_akas.drop(['attributes','ordering','language','types'],axis=1,inplace=True)

# Change title_id To tconst
df_akas.rename({'title_id':'tconst'},axis=1,inplace=True)

### Preparing IMDB Crew Dataset

In [ ]:
# Read In Dataset
df_crew = pd.read_csv('data/title.crew.tsv',delimiter='\t')

# Drop Unneeded Columns
df_crew.drop(['writers'],axis=1,inplace=True)

### Preparing IMDB Names Dataset

In [ ]:
# Read In Dataset
df_names = pd.read_csv('data/imdb.name.basics.csv')

# Drop Unneedeed Columns
df_names.dropna(subset=['primary_profession'],inplace=True)

# Only Keep People That Are Directors
df_names['is_director'] = df_names['primary_profession'].map(identify_director)
df_names = df_names[df_names['is_director'] == True]
df_names = df_names.filter(['nconst','primary_name'],axis=1)
df_names.rename({'nconst':'directors' , 'primary_name':'director_name'},axis=1,inplace=True)

### Merging Datasets

In [ ]:
# Merges
df_genre = df_budget_sorted.merge(df_basics,how='left',on='movie')
df_genre_rating = df_genre.merge(df_ratings,how='left',on='tconst').copy(deep=True)
df_genre_rating_akas = df_genre_rating.merge(df_akas,how='left',on='tconst').copy(deep=True)
df_genre_rating_akas_crew = df_genre_rating_akas.merge(df_crew,how='left',on='tconst').copy(deep=True)


# Drop Duplicates Based On movie Column
df_genre.drop_duplicates('movie' , keep='first' , inplace=True)
df_genre_rating.drop_duplicates('movie' , keep='first' , inplace=True)
df_genre_rating_akas.drop_duplicates('movie' , keep='first' , inplace=True)
df_genre_rating_akas_crew.drop_duplicates('movie' , keep='first' , inplace=True)

## Finding The Most Profitable Genre

### Code

In [ ]:
# Drop Movies Without Genre Data
df_genre.dropna(subset=['genre_lists'],inplace=True)

# Reorder Column Names
column_order = ['tconst','movie','release_year','runtime_minutes','budget_int','total_gross_int','gross_margin','genre_lists','main_genre']
df_genre = df_genre[column_order]

# Keep Only Top 20 main_genres (More Than 30 Movies In Genre Class)
top_20_genres = list(df_genre['main_genre'].value_counts()[:20].index)
df_genre = df_genre[df_genre['main_genre'].isin(top_20_genres)]

# Sort Movies By Profitability
df_genre.sort_values('gross_margin', ascending=False, inplace=True)

# Remove Outliers (Keep Only Movies That Are Within Middle 80%)
median_ = df_genre['gross_margin'].median()
IQR_ = df_genre['gross_margin'].quantile(.90) - df_genre['gross_margin'].quantile(.10)
df_genre = df_genre[(df_genre['gross_margin'] >= median_ - IQR_) & (df_genre['gross_margin'] <= median_ + IQR_)]

# Get The Standard Deviation For Movies Made 2000 or Later By Genre
df_genre_grouped_std = df_genre[df_genre['release_year'] >= 2000].groupby('main_genre').std().filter(['main_genre','gross_margin'])
df_genre_grouped_std.rename({'gross_margin':'gm%_std'},inplace=True,axis=1)

# Get The Min Gross Margin For Movies Made 2000 or Later By Genre
df_genre_grouped_min = df_genre[df_genre['release_year'] >= 2000].groupby('main_genre').min().filter(['main_genre','gross_margin'])
df_genre_grouped_min.rename({'gross_margin':'gm%_min'},inplace=True,axis=1)

# Get The Max Gross Margin For Movies Made 2000 or Later By Genre
df_genre_grouped_max = df_genre[df_genre['release_year'] >= 2000].groupby('main_genre').max().filter(['main_genre','gross_margin'])
df_genre_grouped_max.rename({'gross_margin':'gm%_max'},inplace=True,axis=1)

#Get The Mean Gross Margins For Movies Made 2000 Or Later By Genre
df_genre_grouped = df_genre[df_genre['release_year'] >= 2000].groupby('main_genre').mean().filter(['main_genre','gross_margin'])

# Add the Min, Median and Max Columns to Main Grouped Dataset
df_genre_final = df_genre_grouped_std.join(df_genre_grouped,on='main_genre')
df_genre_final = df_genre_grouped_min.join(df_genre_final,on='main_genre')
df_genre_final = df_genre_grouped_max.join(df_genre_final,on='main_genre')
df_genre_final = df_genre_final.sort_values('gross_margin',ascending=True)

### Plotting Profitability By Genre

In [ ]:
# Plotting
fig, axes = plt.subplots()
df_genre_final.filter(items=['gm%_min','gross_margin','gm%_max'])[::-1].plot(kind='bar',
                      figsize= (20,12),xlabel='Genre',ylabel = 'Gross Margin (%)',
                      title = 'Profitability Of Top 20 Most Common Movie Genres',
                      ax=axes,label='Medain GM%',color=['Red','Black','Green'])

# plt.ylim(-50,100)
axes.legend(labels=['Lowest Profitable Movie In Genre','Median Profitability of Genre','Highest Profitable Movie In Genre'])
plt.xticks(rotation=60)
plt.show()

### Plotting Median Genre Profitability With Standard Deviation

In [ ]:
plot_ = df_genre_final['gross_margin'][::-1].plot(kind='bar',xlabel='Genre',ylabel = 'Gross Margin (%)',
                                                  title = 'Median Profitability Of Top 20 Most Common Movie Genres',
                                                  yerr=df_genre_final['gm%_std'],figsize=(20,12))

### Plotting Profitability v Movie Budget

In [ ]:
# Keep Only Movies That Are Within Inner 80%
median_r = df_genre_rating_akas['gross_margin'].median()
IQR_r = df_genre_rating_akas['gross_margin'].quantile(.90) - df_genre_rating_akas['gross_margin'].quantile(.10)
df_genre_rating_akas = df_genre_rating_akas[(df_genre_rating_akas['gross_margin'] >= median_r - IQR_r) & (df_genre_rating_akas['gross_margin'] <= median_r + IQR_r)].copy(deep=True)

# Add Gross Margin Dollars Column
df_genre_rating_akas["gross_margin_dollars"] = df_genre_rating_akas['total_gross_int'] - df_genre_rating_akas['budget_int']

In [ ]:
# Plotting
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_figheight(12)
fig.set_figwidth(20)

horror_list = ['Horror/Mystery']
adventure_list = ['Adventure/Comedy']
comedy_list = ['Comedy/Romance']
action_list = ['Action/Adventure']

df_genre_rating_akas[df_genre_rating_akas['release_year'] >= 2000].plot(ax=axes[0,0],kind='scatter' , x='budget_int' , y ='gross_margin',layout=(2,2))
df_genre_rating_akas[(df_genre_rating_akas['release_year'] >= 2000) & (df_genre_rating_akas['main_genre'].isin(horror_list))].plot(ax=axes[0,0],kind='scatter',x='budget_int',y='gross_margin',color='DarkOrange',label='Horror/Mystery')

df_genre_rating_akas[df_genre_rating_akas['release_year'] >= 2000].plot(ax=axes[0,1],kind='scatter' , x='budget_int' , y ='gross_margin',layout=(2,2))
df_genre_rating_akas[(df_genre_rating_akas['release_year'] >= 2000) & (df_genre_rating_akas['main_genre'].isin(adventure_list))].plot(ax=axes[0,1],kind='scatter',x='budget_int',y='gross_margin',color='DarkGreen',label='Adventure/Comedy')

df_genre_rating_akas[df_genre_rating_akas['release_year'] >= 2000].plot(ax=axes[1,0],kind='scatter' , x='budget_int' , y ='gross_margin',layout=(2,2))
df_genre_rating_akas[(df_genre_rating_akas['release_year'] >= 2000) & (df_genre_rating_akas['main_genre'].isin(comedy_list))].plot(ax=axes[1,0],kind='scatter',x='budget_int',y='gross_margin',color='DarkRed',label='Comedy/Romance')

df_genre_rating_akas[df_genre_rating_akas['release_year'] >= 2000].plot(ax=axes[1,1],kind='scatter' , x='budget_int' , y ='gross_margin',layout=(2,2))
df_genre_rating_akas[(df_genre_rating_akas['release_year'] >= 2000) & (df_genre_rating_akas['main_genre'].isin(action_list))].plot(ax=axes[1,1],kind='scatter',x='budget_int',y='gross_margin',color='Yellow',label='Action/Adventure')

fig.suptitle('Profitability (%GM) vs Movie Budget (USD)',fontsize=20)
for i in range(2):
    for j in range(2):
        axes[i,j].set_ylabel('Profitability (%GM)')
        axes[i,j].set_xlabel('Movie Budget (USD)')
plt.show()

### Example Horror/Mystery Movies

In [ ]:
# Drop Movies With No Info On Directors
df_genre_rating_akas_crew.dropna(subset=['directors'],inplace=True)

# Swap Director ID For Director Name
df_director = df_genre_rating_akas_crew.merge(df_names,how='left',on='directors').copy(deep=True)
df_director.dropna(subset=['director_name'],inplace=True)

# Filter Movies For Only Horror/Mystery
horror_list = ['Horror/Mystery']
df_director = df_director[df_director['main_genre'].isin(horror_list)]

In [ ]:
# Cleanup df_director (Keep Relevant Columns)
df_director_final = df_director.sort_values(['total_gross_int','gross_margin'],ascending=False).head(20).filter(['movie','budget_int','total_gross_int','gross_margin','release_year','averageRating','director_name']).copy(deep=True)

# Change Money Back To str In USD
df_director_final['budget_int'] = df_director_final['budget_int'].map(dirty_money)
df_director_final['total_gross_int'] = df_director_final['total_gross_int'].map(dirty_money)

# Clean Up Column Names For Readability
df_director_final.columns = ['Movie Title', 'Budget (USD)', 'Total Gross (USD)', 'Profitability (GM%)', 'Release Year', 'Average IMDB Rating', 'Director']

In [ ]:
df_director_final.head(10)